**Membuat Model Untuk Klasifikasi Dua Kelas**

Saya akan mencoba melatih model untuk mengklasifikasikan apakah sebuah buah merupakan jeruk atau anggur

pertama-tama kita import library pandas untuk mengextract dataset menjadi dataframe

In [1]:
import pandas as pd

df = pd.read_csv('citrus.csv')

Untuk melihat overview dataset kita bisa menggunakan fungsi info()

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      10000 non-null  object 
 1   diameter  10000 non-null  float64
 2   weight    10000 non-null  float64
 3   red       10000 non-null  int64  
 4   green     10000 non-null  int64  
 5   blue      10000 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 468.9+ KB


untuk menampilkan dataframe kita bisa menggunakan fungsi head()

In [4]:
df.head(10000)

,name,diameter,weight,red,green,blue
0,orange,2.96,86.76,172,85,2
1,orange,3.91,88.05,166,78,3
2,orange,4.42,95.17,156,81,2
3,orange,4.47,95.60,163,81,4
4,orange,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,grapefruit,15.35,253.89,149,77,20
9996,grapefruit,15.41,254.67,148,68,7
9997,grapefruit,15.59,256.50,168,82,20
9998,grapefruit,15.92,260.14,142,72,11


Jaringan saraf tiruan tidak bisa memproses string, maka dari itu kita harus mengubah nilai-nilai pada kolom label numerik terlebih dahulu

pada kode dibawah string 'orange' dirubah menjadi 0 dan 'grapefruit' dirubah menjadi 1

In [5]:
df.name[df.name == 'orange'] = 0
df.name[df.name == 'grapefruit'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


pada proses ini kita masih belum bisa memproses dataset, karena dataset kita masih dalam bentuk dataframe. Dataset harus dalam bentuk array agar dapat diproses oleh model

In [7]:
dataset = df.values

Untuk melihat dataframe yang telah diubah menjadi array, kita bisa memanggil variabel tersebut

In [8]:
dataset

array([[0, 2.96, 86.76, 172, 85, 2],
       [0, 3.91, 88.05, 166, 78, 3],
       [0, 4.42, 95.17, 156, 81, 2],
       ...,
       [1, 15.59, 256.5, 168, 82, 20],
       [1, 15.92, 260.14, 142, 72, 11],
       [1, 16.45, 261.51, 152, 74, 2]], dtype=object)

Kita perlu memisahkan antara atribut dan label pada dataset. Dapat diketahui 4 kolom terakhir adalah kolom atribut dan pada kolom pertama adalah label

In [9]:
# Pilih 4 kolom terakhir sebagai atribut

X = dataset[:,1:6]

# Bilangan sebelum koma untuk memilih baris pada dataframe
# Bilangan setelah koma untuk memilih kolom pada dataframe

y = dataset[:,0]

tahap yang penting dilakukan agar jaringan saraf tiruan bisa mempelajari dataset dengan baik adalah dengan normalisasi. Kita bisa menggunakan fungsi fit_transform() dari sebuah objek MinMaxScaler dari library preprocessing SKLearn untuk menormalisasi data

In [10]:
# Normalization

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

array([[0.        , 0.        , 0.74025974, 0.63529412, 0.        ],
       [0.07042254, 0.00738197, 0.66233766, 0.55294118, 0.01851852],
       [0.10822832, 0.04812589, 0.53246753, 0.58823529, 0.        ],
       ...,
       [0.93624907, 0.97133047, 0.68831169, 0.6       , 0.33333333],
       [0.96071164, 0.99216023, 0.35064935, 0.48235294, 0.16666667],
       [1.        , 1.        , 0.48051948, 0.50588235, 0.        ]])

Setelah atribut dan label terpisah, kita juga akan memisahkan kembali data kita menjadi data latih dan data uji untuk mengevaluasi kinerja model kita. Fungsi train_test_split dapat membantu kita untuk melakukan hal ini dengan mudah. Pada latihan ini kita akan menggunakan 30% dari seluruh data uji

In [11]:
# Pisahkan data training dan testing

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_scale, y, test_size=0.3)

Dataset kini sudah dipisahkan ke dalam training dan test set. Karena tadi kita mengubah nilai-nilai pada kolom name menjadi bilangan numerik, artinya kita mengubah label menjadi tipe data boolean, maka kita perlu mengubah tipe data tersebut menjadi float32

In [12]:
import numpy as np

Y_train = Y_train.astype(np.float32)
Y_test = Y_test.astype(np.float32)

Untuk model yang kita kembangkan adalah model sequential yang memiliki 3 buah layer. Activation function pada 2 layer pertama yang dapat digunakan adalah relu untuk latihan ini. Untuk layer terakhir, isi parameter unit isi dengan 1 di mana output dari jaringan saraf tiruan ktia merupakan satu buah bilangan numerik. Activation function pada layer terakhir dipilih sigmoid karena sigmoid memetakan probabilitas dari 0 sampai 1. Sigmoid cocok digunakan untuk masalah klasifikasi biner

In [13]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
                    Dense(32, activation='relu', input_shape=(5,)),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid')
])

Setelah arsitektur dari jaringan saraf tiruan dibentuk, kita perlu menentukan optimizer dan loss function dari model kita. Untuk optimizer kita menggunakan stochastic gradient descent (sgd) yang cocok untuk dataset berukuran kecil dan untuk loss menggunakan 'binary_crossentropy'

Selain itu kita akan menambahkan parameter accuracy agar menampilkan akurasi pada setiap proses pelatihan model

In [14]:
model.compile(
    optimizer='sgd',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

Ketika semua sudah selesai kita bisa memulai pelatihan model

In [15]:
model.fit(X_train, Y_train, epochs=300)

Epoch 1/300
219/219 [==============================] - 15s 1ms/step - loss: 0.6749 - accuracy: 0.7875
Epoch 2/300
219/219 [==============================] - 0s 1ms/step - loss: 0.6244 - accuracy: 0.9141
Epoch 3/300
219/219 [==============================] - 0s 1ms/step - loss: 0.5576 - accuracy: 0.9127
Epoch 4/300
219/219 [==============================] - 0s 1ms/step - loss: 0.4751 - accuracy: 0.9132
Epoch 5/300
219/219 [==============================] - 0s 1ms/step - loss: 0.3756 - accuracy: 0.9196
Epoch 6/300
219/219 [==============================] - 0s 1ms/step - loss: 0.3044 - accuracy: 0.9253
Epoch 7/300
219/219 [==============================] - 0s 1ms/step - loss: 0.2521 - accuracy: 0.9274
Epoch 8/300
219/219 [==============================] - 0s 1ms/step - loss: 0.2286 - accuracy: 0.9196
Epoch 9/300
219/219 [==============================] - 0s 1ms/step - loss: 0.2154 - accuracy: 0.9219
Epoch 10/300
219/219 [==============================] - 0s 1ms/step - loss: 0.1974 - accur

In [16]:
model.evaluate(X_test, Y_test)
# Elemen pertama adalah loss dan elemen kedua adalah akurasi

94/94 [==============================] - 1s 1ms/step - loss: 0.1668 - accuracy: 0.9290


[0.16682958602905273, 0.9290000200271606]